# Topic Analysis of CRINK Voting Patterns

This notebook generates topic-level analysis of CRINK (China, Russia, Iran, North Korea) voting alignment in UN General Assembly resolutions.

## Workflow
1. Load UN voting data
2. Generate or load embeddings for resolution titles
3. Run HDBSCAN clustering on embeddings
4. Apply confirmed topic mappings (using fuzzy matching to reuse existing mappings)
5. Assign noise resolutions to meta-topics (using undl_id-based mapping)
6. Calculate CRINK alignment metrics by topic
7. Generate summary tables for publication

## Embedding Generation
If embeddings don't exist or are mismatched, this notebook will generate new embeddings using the OpenAI API (requires `OPEN_AI_API` environment variable).

## Reusing Existing Mappings
- **Topic mappings**: Uses fuzzy matching to map new cluster labels to existing meta-topics
- **Noise mappings**: Uses `undl_id` directly - works for any overlapping resolutions

**Files required:**
- `data/processed/UNGA_voting_records_filtered.csv`
- `data/mappings/topic_mapping_confirmed_*.json`
- `data/mappings/noise_mapping_confirmed_*.json`

**Files generated (if needed):**
- `data/mappings/UNGA_embeddings.npy`
- `data/mappings/UNGA_embeddings_meta.json`

In [1]:
# Cell 1: Imports and Configuration
import os
import pandas as pd
import numpy as np
import json
import glob
import re
from pathlib import Path
from datetime import datetime
from collections import Counter
from typing import List
import hdbscan
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# ================================================================
# CONFIGURATION
# ================================================================

# CRINK country definitions
CRINK_COUNTRIES = {
    'CHINA': 'CHN',
    'RUSSIAN FEDERATION': 'RUS',
    'IRAN (ISLAMIC REPUBLIC OF)': 'IRN',
    "DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA": 'PRK'
}

# HDBSCAN parameters
HDBSCAN_MIN_CLUSTER_SIZE = 10
HDBSCAN_MIN_SAMPLES = 1

# OpenAI embedding model
EMBEDDING_MODEL = "text-embedding-3-large"
EMBEDDING_BATCH_SIZE = 128

# Plot and output settings
FIGURE_SIZE = (14, 5)
DPI = 300

# Set up paths
notebook_dir = Path.cwd()
repo_root = notebook_dir.parent
data_dir = repo_root / 'data' / 'processed'
mappings_dir = repo_root / 'data' / 'mappings'
results_dir = repo_root / 'results'

# Embedding file paths
embeddings_file = 'UNGA_embeddings.npy'
embeddings_meta_file = 'UNGA_embeddings_meta.json'
embeddings_path = mappings_dir / embeddings_file
meta_path = mappings_dir / embeddings_meta_file

# Ensure directories exist
results_dir.mkdir(parents=True, exist_ok=True)
mappings_dir.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Mappings directory: {mappings_dir}")
print(f"Results directory: {results_dir}")
print("\nConfiguration loaded successfully")

Data directory: c:\Users\Lucian\OneDrive - Tulane University\01 IFSH\Data Sciences try\measuring-CRINK-alignment-UN\data\processed
Mappings directory: c:\Users\Lucian\OneDrive - Tulane University\01 IFSH\Data Sciences try\measuring-CRINK-alignment-UN\data\mappings
Results directory: c:\Users\Lucian\OneDrive - Tulane University\01 IFSH\Data Sciences try\measuring-CRINK-alignment-UN\results

Configuration loaded successfully


In [2]:
# Cell 2: Helper Functions

def load_mapping_json(filepath):
    """Load mapping dictionary from JSON file."""
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Separate metadata from mappings
    metadata = data.pop('_metadata', {})
    return data, metadata


def find_latest_mapping_file(mappings_dir, pattern):
    """Find the most recent mapping file matching a pattern."""
    matching_files = sorted(glob.glob(str(mappings_dir / pattern)))
    if not matching_files:
        return None
    return matching_files[-1]  # Most recent by timestamp in filename


def clean_text(text):
    """Minimal text cleaning for embeddings."""
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def fuzzy_match_topic(new_label, existing_mappings, threshold=0.5):
    """
    Try to match a new topic label to existing mappings using word overlap.
    Returns (meta_topic, confidence) or (None, 0) if no match.
    """
    new_words = set(new_label.lower().split())
    
    best_match = None
    best_score = 0
    
    for old_label, meta_topic in existing_mappings.items():
        old_words = set(old_label.lower().split())
        
        # Jaccard similarity
        if len(new_words | old_words) > 0:
            score = len(new_words & old_words) / len(new_words | old_words)
            if score > best_score:
                best_score = score
                best_match = meta_topic
    
    if best_score >= threshold:
        return best_match, best_score
    return None, 0


print("Helper functions loaded")

Helper functions loaded


In [3]:
# Cell 3: Load Voting Data

csv_file = 'UNGA_voting_records_filtered.csv'
dataset_type = 'all_resolutions'

# Year filter for post-Cold War analysis
START_YEAR = 1991

csv_path = data_dir / csv_file

if not csv_path.exists():
    raise FileNotFoundError(
        f"Data file not found: {csv_path}\n"
        f"Please download from Harvard Dataverse and place in {data_dir}"
    )

# Load data
df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)

# Standardize country names
if 'ms_name' in df.columns:
    df['ms_name'] = df['ms_name'].replace({'USSR': 'RUSSIAN FEDERATION'})

# Convert dates and extract year
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['year'] = df['date'].dt.year

# Filter to post-Cold War era (1991+)
df = df[df['year'] >= START_YEAR].copy()
print(f"✓ Filtered to {START_YEAR}+ (post-Cold War era)")

# Create resolution-level dataframe
resolution_df = df.drop_duplicates('undl_id')[['undl_id', 'title', 'date', 'year']].copy()
resolution_df = resolution_df.reset_index(drop=True)
resolution_df['text_clean'] = resolution_df['title'].apply(clean_text)

print(f"✓ Loaded {csv_file}")
print(f"  Total rows: {len(df):,}")
print(f"  Unique resolutions: {len(resolution_df):,}")
print(f"  Year range: {resolution_df['year'].min()}-{resolution_df['year'].max()}")

✓ Filtered to 1991+ (post-Cold War era)
✓ Loaded UNGA_voting_records_filtered.csv
  Total rows: 492,762
  Unique resolutions: 2,597
  Year range: 1991-2024


In [4]:
# Cell 4: Generate or Load Embeddings

from dotenv import load_dotenv

# Load .env file from repo root
env_path = repo_root / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print(f"✓ Loaded .env from {env_path}")
else:
    print(f"⚠ No .env file found at {env_path}")


def get_embeddings_from_api(texts: List[str], client) -> np.ndarray:
    """Call OpenAI embeddings API for a list of texts."""
    response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=texts
    )
    return np.array([item.embedding for item in response.data], dtype="float32")


def generate_embeddings(texts: List[str]) -> np.ndarray:
    """Generate embeddings using OpenAI API."""
    from openai import OpenAI
    
    api_key = os.getenv("OPEN_AI_API")
    if not api_key:
        raise ValueError(
            "OpenAI API key not found.\n"
            "Either:\n"
            "  1. Create a .env file in repo root with: OPEN_AI_API=your-key\n"
            "  2. Or set environment variable: $env:OPEN_AI_API = 'your-key'"
        )
    
    client = OpenAI(api_key=api_key)
    all_embeddings = []
    n = len(texts)
    
    for start in range(0, n, EMBEDDING_BATCH_SIZE):
        end = min(start + EMBEDDING_BATCH_SIZE, n)
        batch = texts[start:end]
        print(f"  Embedding batch {start}-{end} of {n}...")
        batch_embeddings = get_embeddings_from_api(batch, client)
        all_embeddings.append(batch_embeddings)
    
    return np.vstack(all_embeddings)


# Check if embeddings exist and match resolution count
need_regenerate = False
n_resolutions = len(resolution_df)

if embeddings_path.exists() and meta_path.exists():
    # Load existing embeddings
    embeddings = np.load(embeddings_path)
    with open(meta_path, 'r') as f:
        emb_meta = json.load(f)
    
    print(f"Found existing embeddings: {embeddings.shape}")
    print(f"  Model: {emb_meta.get('model', 'unknown')}")
    print(f"  Expected rows: {emb_meta.get('n_rows', 'unknown')}")
    
    # Check if it matches current data
    if embeddings.shape[0] != n_resolutions:
        print(f"\n⚠ Mismatch: embeddings have {embeddings.shape[0]} rows, data has {n_resolutions} resolutions")
        need_regenerate = True
else:
    print("No existing embeddings found.")
    need_regenerate = True

if need_regenerate:
    print(f"\n🔄 Generating new embeddings for {n_resolutions} resolutions...")
    texts = resolution_df['text_clean'].tolist()
    embeddings = generate_embeddings(texts)
    
    # Save embeddings
    np.save(embeddings_path, embeddings)
    
    # Save metadata
    emb_meta = {
        "model": EMBEDDING_MODEL,
        "n_rows": n_resolutions,
        "text_column": "title",
        "generated_at": datetime.now().isoformat()
    }
    with open(meta_path, 'w') as f:
        json.dump(emb_meta, f, indent=2)
    
    print(f"\n✓ Saved new embeddings: {embeddings.shape}")
    print(f"  File: {embeddings_path.name}")
else:
    print("\n✓ Using existing embeddings")

✓ Loaded .env from c:\Users\Lucian\OneDrive - Tulane University\01 IFSH\Data Sciences try\measuring-CRINK-alignment-UN\.env
Found existing embeddings: (7812, 3072)
  Model: text-embedding-3-large
  Expected rows: 7812

⚠ Mismatch: embeddings have 7812 rows, data has 2597 resolutions

🔄 Generating new embeddings for 2597 resolutions...
  Embedding batch 0-128 of 2597...
  Embedding batch 128-256 of 2597...
  Embedding batch 256-384 of 2597...
  Embedding batch 384-512 of 2597...
  Embedding batch 512-640 of 2597...
  Embedding batch 640-768 of 2597...
  Embedding batch 768-896 of 2597...
  Embedding batch 896-1024 of 2597...
  Embedding batch 1024-1152 of 2597...
  Embedding batch 1152-1280 of 2597...
  Embedding batch 1280-1408 of 2597...
  Embedding batch 1408-1536 of 2597...
  Embedding batch 1536-1664 of 2597...
  Embedding batch 1664-1792 of 2597...
  Embedding batch 1792-1920 of 2597...
  Embedding batch 1920-2048 of 2597...
  Embedding batch 2048-2176 of 2597...
  Embedding batch

In [5]:
# Cell 5: Run HDBSCAN Clustering

print("Running HDBSCAN clustering on embeddings...")

# Create and fit HDBSCAN with same parameters as original
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
    min_samples=HDBSCAN_MIN_SAMPLES,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

cluster_labels = clusterer.fit_predict(embeddings)

# Attach to dataframe
resolution_df['cluster'] = cluster_labels
resolution_df['cluster_prob'] = clusterer.probabilities_

n_clusters = len(set(cluster_labels) - {-1})
n_noise = (cluster_labels == -1).sum()

print(f"\n✓ HDBSCAN clustering complete")
print(f"  Clusters found: {n_clusters}")
print(f"  Noise points: {n_noise} ({100*n_noise/len(resolution_df):.1f}%)")
print(f"\nCluster distribution:")
print(resolution_df['cluster'].value_counts().head(10))

Running HDBSCAN clustering on embeddings...

✓ HDBSCAN clustering complete
  Clusters found: 98
  Noise points: 533 (20.5%)

Cluster distribution:
cluster
-1     533
 90     42
 79     41
 21     38
 7      37
 37     37
 59     35
 26     35
 42     33
 40     33
Name: count, dtype: int64


In [6]:
# Cell 6: Generate Cluster Topic Labels

# For each cluster, create a simple label based on common words in titles
# (This is a reproducible fallback; the confirmed mappings will override these)

def get_cluster_label(cluster_id, df, text_col='title', max_examples=10):
    """Generate a simple label for a cluster based on common title words."""
    if cluster_id == -1:
        return "Noise / Unclustered"
    
    cluster_texts = df[df['cluster'] == cluster_id][text_col].head(max_examples).tolist()
    if not cluster_texts:
        return f"Cluster {cluster_id}"
    
    # Simple word frequency approach
    words = []
    stop_words = {'the', 'of', 'and', 'to', 'in', 'a', 'for', 'on', 'by', 'with', 'its', 'an', 'as', 'at', 'from'}
    for text in cluster_texts:
        words.extend([w.lower() for w in str(text).split() if w.lower() not in stop_words and len(w) > 2])
    
    word_counts = Counter(words)
    top_words = [w for w, c in word_counts.most_common(4)]
    return ' '.join(top_words).title() if top_words else f"Cluster {cluster_id}"


# Generate initial topic labels for all clusters
cluster_ids = sorted(set(resolution_df['cluster']))
cluster_topic_labels = {}

for cl in cluster_ids:
    cluster_topic_labels[cl] = get_cluster_label(cl, resolution_df)

# Apply labels to dataframe
resolution_df['topic_label'] = resolution_df['cluster'].map(cluster_topic_labels)

print(f"✓ Generated topic labels for {len(cluster_topic_labels)} clusters")
print(f"\nSample labels:")
for cl in list(cluster_ids)[:5]:
    count = (resolution_df['cluster'] == cl).sum()
    print(f"  Cluster {cl:3d} ({count:4d} items): {cluster_topic_labels[cl]}")

✓ Generated topic labels for 99 clusters

Sample labels:
  Cluster  -1 ( 533 items): Noise / Unclustered
  Cluster   0 (  18 items): Oil Slick Lebanese Shores
  Cluster   1 (  11 items): Extrajudicial, Summary Arbitrary Executions
  Cluster   2 (  11 items): Respect Universal Freedom Travel
  Cluster   3 (  23 items): American Islands Samoa Guam


In [7]:
# Cell 7: Load Confirmed Topic Mapping and Apply with Fuzzy Matching

# Find the most recent confirmed topic mapping file
topic_mapping_pattern = f"topic_mapping_confirmed_{dataset_type}_*.json"
topic_mapping_file = find_latest_mapping_file(mappings_dir, topic_mapping_pattern)

if topic_mapping_file is None:
    raise FileNotFoundError(
        f"No topic mapping file found matching: {topic_mapping_pattern}\n"
        f"Expected in: {mappings_dir}"
    )

topic_to_meta, topic_meta = load_mapping_json(topic_mapping_file)

print(f"✓ Loaded topic mapping: {Path(topic_mapping_file).name}")
print(f"  Original mappings: {len(topic_to_meta)}")

# Get available meta-topics
available_meta_topics = sorted(set(topic_to_meta.values()))
print(f"  Meta-topics: {len(available_meta_topics)}")

# Map new cluster labels to meta-topics using fuzzy matching
new_topic_to_meta = {}
unmatched_topics = []
match_stats = {'exact': 0, 'fuzzy': 0, 'unmatched': 0}

for cluster_id, new_label in cluster_topic_labels.items():
    if cluster_id == -1:
        continue  # Skip noise cluster
    
    # Try exact match first
    if new_label in topic_to_meta:
        new_topic_to_meta[new_label] = topic_to_meta[new_label]
        match_stats['exact'] += 1
    else:
        # Try fuzzy match
        meta_topic, confidence = fuzzy_match_topic(new_label, topic_to_meta, threshold=0.3)
        if meta_topic:
            new_topic_to_meta[new_label] = meta_topic
            match_stats['fuzzy'] += 1
        else:
            # No match - assign to Miscellaneous
            new_topic_to_meta[new_label] = 'Miscellaneous'
            unmatched_topics.append(new_label)
            match_stats['unmatched'] += 1

print(f"\nMapping results:")
print(f"  Exact matches: {match_stats['exact']}")
print(f"  Fuzzy matches: {match_stats['fuzzy']}")
print(f"  Unmatched (→ Miscellaneous): {match_stats['unmatched']}")

if unmatched_topics:
    print(f"\nUnmatched topics (assigned to Miscellaneous):")
    for t in unmatched_topics[:10]:
        print(f"  - {t}")
    if len(unmatched_topics) > 10:
        print(f"  ... and {len(unmatched_topics) - 10} more")

✓ Loaded topic mapping: topic_mapping_confirmed_all_resolutions_20251208_180017.json
  Original mappings: 86
  Meta-topics: 9

Mapping results:
  Exact matches: 1
  Fuzzy matches: 40
  Unmatched (→ Miscellaneous): 57

Unmatched topics (assigned to Miscellaneous):
  - Oil Slick Lebanese Shores
  - Extrajudicial, Summary Arbitrary Executions
  - Respect Universal Freedom Travel
  - American Islands Samoa Guam
  - Measures Uphold Authority 1925
  - Financing United Nations Interim
  - Law Sea Oceans Agreement
  - Necessity Ending Economic, Commercial
  - Rights Use Mercenaries Means
  - Equitable Geographical Distribution Membership
  ... and 47 more


In [8]:
# Cell 8: Load Confirmed Noise Mapping (by undl_id)

# Find the most recent confirmed noise mapping file
noise_mapping_pattern = f"noise_mapping_confirmed_{dataset_type}_*.json"
noise_mapping_file = find_latest_mapping_file(mappings_dir, noise_mapping_pattern)

if noise_mapping_file is None:
    print(f"⚠ No noise mapping file found. Noise items will be labeled 'Miscellaneous'.")
    noise_to_meta = {}
else:
    noise_mapping_raw, noise_meta = load_mapping_json(noise_mapping_file)
    
    # Handle both formats:
    # Format 1: {"undl_id": "meta_topic", ...}
    # Format 2: {"0": {"id": "undl_id", "assigned_topic": "meta_topic"}, ...}
    noise_to_meta = {}
    for key, value in noise_mapping_raw.items():
        if isinstance(value, dict) and 'id' in value and 'assigned_topic' in value:
            # Format 2
            noise_to_meta[str(value['id'])] = value['assigned_topic']
        elif isinstance(value, str):
            # Format 1
            noise_to_meta[str(key)] = value
    
    print(f"✓ Loaded noise mapping: {Path(noise_mapping_file).name}")
    print(f"  Noise resolutions mapped: {len(noise_to_meta)}")

# Count how many noise points we can map
noise_resolutions = resolution_df[resolution_df['cluster'] == -1]['undl_id'].astype(str).tolist()
mappable = sum(1 for uid in noise_resolutions if uid in noise_to_meta)
print(f"\n  Current noise points: {len(noise_resolutions)}")
print(f"  Mappable from existing: {mappable} ({100*mappable/max(1,len(noise_resolutions)):.1f}%)")

✓ Loaded noise mapping: noise_mapping_confirmed_all_resolutions_20251208_180624.json
  Noise resolutions mapped: 507

  Current noise points: 533
  Mappable from existing: 176 (33.0%)


In [9]:
# Cell 9: Apply Mappings to Create Meta-Topic Column

print("Applying topic and noise mappings...")

# Initialize meta_topic_label column
resolution_df['meta_topic_label'] = None

# Step 1: Apply topic-to-meta mapping for clustered (non-noise) rows
mapped_from_topic = 0
for new_label, meta_topic in new_topic_to_meta.items():
    mask = resolution_df['topic_label'] == new_label
    resolution_df.loc[mask, 'meta_topic_label'] = meta_topic
    mapped_from_topic += mask.sum()

print(f"  Mapped from topic labels: {mapped_from_topic}")

# Step 2: Apply noise-to-meta mapping for noise rows (by undl_id)
mapped_from_noise = 0
if noise_to_meta:
    for idx, row in resolution_df[resolution_df['cluster'] == -1].iterrows():
        undl_id_str = str(row['undl_id'])
        if undl_id_str in noise_to_meta:
            resolution_df.loc[idx, 'meta_topic_label'] = noise_to_meta[undl_id_str]
            mapped_from_noise += 1

print(f"  Mapped from noise (by undl_id): {mapped_from_noise}")

# Step 3: Fill remaining nulls with 'Miscellaneous'
unmapped = resolution_df['meta_topic_label'].isna().sum()
resolution_df['meta_topic_label'] = resolution_df['meta_topic_label'].fillna('Miscellaneous')
print(f"  Assigned to Miscellaneous: {unmapped}")

# Create numeric meta_topic_id
unique_meta_topics = sorted(resolution_df['meta_topic_label'].unique())
meta_topic_id_map = {mt: i for i, mt in enumerate(unique_meta_topics)}
resolution_df['meta_topic_id'] = resolution_df['meta_topic_label'].map(meta_topic_id_map)

print(f"\n✓ Applied mappings to {len(resolution_df)} resolutions")
print(f"  Unique meta-topics: {len(unique_meta_topics)}")
print(f"\nMeta-topic distribution:")
print(resolution_df['meta_topic_label'].value_counts())

Applying topic and noise mappings...
  Mapped from topic labels: 2064
  Mapped from noise (by undl_id): 176
  Assigned to Miscellaneous: 357

✓ Applied mappings to 2597 resolutions
  Unique meta-topics: 10

Meta-topic distribution:
meta_topic_label
Miscellaneous                1511
Disarmament                   315
Israel/Palestine Conflict     298
Human Rights                  202
Development                   146
Displacement                   61
International Cooperation      50
International Law               9
Human Rights in CRINK           4
Economic Sanctions              1
Name: count, dtype: int64


In [10]:
# Cell 10: Calculate CRINK Voting Alignment by Resolution

print("Calculating CRINK voting alignment...")

# Get CRINK country votes for each resolution
crink_names = list(CRINK_COUNTRIES.keys())
crink_codes = list(CRINK_COUNTRIES.values())

# Pivot to get one column per CRINK country
crink_votes_df = df[df['ms_name'].isin(crink_names)].pivot_table(
    index='undl_id',
    columns='ms_name',
    values='ms_vote',
    aggfunc='first'
).reset_index()

# Merge with resolution_df
resolution_df = resolution_df.merge(crink_votes_df, on='undl_id', how='left')

# Calculate alignment metrics
def calculate_crink_alignment(row):
    """Calculate how many CRINK countries voted together."""
    votes = [row.get(c) for c in crink_names if c in row.index and pd.notna(row.get(c))]
    
    if len(votes) < 2:
        return pd.Series({
            'crink_votes_available': len(votes),
            'max_agreement_count': 0,
            'four_vote_alignment': 0,
            'three_vote_alignment': 0,
            'two_vote_alignment': 0,
            'crink_group_vote': None
        })
    
    vote_counts = Counter(votes)
    most_common_vote, max_count = vote_counts.most_common(1)[0]
    
    return pd.Series({
        'crink_votes_available': len(votes),
        'max_agreement_count': max_count,
        'four_vote_alignment': 1 if max_count == 4 else 0,
        'three_vote_alignment': 1 if max_count == 3 else 0,
        'two_vote_alignment': 1 if max_count == 2 else 0,
        'crink_group_vote': most_common_vote
    })

alignment_results = resolution_df.apply(calculate_crink_alignment, axis=1)
resolution_df = pd.concat([resolution_df, alignment_results], axis=1)

print(f"\n✓ Calculated alignment for {len(resolution_df)} resolutions")
print(f"\nAlignment summary:")
print(f"  4-way alignment (all 4 CRINK): {resolution_df['four_vote_alignment'].sum()}")
print(f"  3-way alignment (3 of 4 CRINK): {resolution_df['three_vote_alignment'].sum()}")
print(f"  2-way alignment (2 of 4 CRINK): {resolution_df['two_vote_alignment'].sum()}")

Calculating CRINK voting alignment...

✓ Calculated alignment for 2597 resolutions

Alignment summary:
  4-way alignment (all 4 CRINK): 1373
  3-way alignment (3 of 4 CRINK): 793
  2-way alignment (2 of 4 CRINK): 401


In [11]:
# Cell 11: Add US Vote and UN Majority Alignment

# Get US votes
us_votes = df[df['ms_name'] == 'UNITED STATES'][['undl_id', 'ms_vote']].rename(
    columns={'ms_vote': 'us_vote'}
).drop_duplicates('undl_id')

resolution_df = resolution_df.merge(us_votes, on='undl_id', how='left')

# Calculate CRINK against US
resolution_df['crink_against_us'] = (
    (resolution_df['crink_group_vote'] != resolution_df['us_vote']) &
    resolution_df['crink_group_vote'].notna() &
    resolution_df['us_vote'].notna() &
    (resolution_df['max_agreement_count'] >= 2)
).astype(int)

# Calculate UN majority vote per resolution
def get_majority_vote(group):
    votes = group['ms_vote'].dropna()
    if len(votes) == 0:
        return None
    return votes.mode().iloc[0] if len(votes.mode()) > 0 else None

majority_votes = df.groupby('undl_id').apply(get_majority_vote).reset_index()
majority_votes.columns = ['undl_id', 'un_majority_vote']

resolution_df = resolution_df.merge(majority_votes, on='undl_id', how='left')

# Calculate CRINK with majority
resolution_df['crink_with_majority'] = (
    (resolution_df['crink_group_vote'] == resolution_df['un_majority_vote']) &
    resolution_df['crink_group_vote'].notna() &
    (resolution_df['max_agreement_count'] >= 2)
).astype(int)

print(f"✓ Added US vote and UN majority alignment")
print(f"  CRINK against US: {resolution_df['crink_against_us'].sum()} resolutions")
print(f"  CRINK with UN majority: {resolution_df['crink_with_majority'].sum()} resolutions")

✓ Added US vote and UN majority alignment
  CRINK against US: 2219 resolutions
  CRINK with UN majority: 2117 resolutions


C:\Users\Lucian\AppData\Local\Temp\ipykernel_22520\3287003252.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  majority_votes = df.groupby('undl_id').apply(get_majority_vote).reset_index()


In [12]:
# Cell 12: Generate Topic Alignment Summary Table

print("Generating topic alignment summary table...\n")

# Aggregate by meta-topic
topic_summary = resolution_df.groupby('meta_topic_label').agg(
    n_resolutions=('undl_id', 'count'),
    four_way_count=('four_vote_alignment', 'sum'),
    three_way_count=('three_vote_alignment', 'sum'),
    two_way_count=('two_vote_alignment', 'sum'),
    against_us_count=('crink_against_us', 'sum'),
    with_majority_count=('crink_with_majority', 'sum')
).reset_index()

# Calculate percentages
topic_summary['four_way_pct'] = (100 * topic_summary['four_way_count'] / topic_summary['n_resolutions']).round(1)
topic_summary['three_way_pct'] = (100 * topic_summary['three_way_count'] / topic_summary['n_resolutions']).round(1)
topic_summary['two_way_pct'] = (100 * topic_summary['two_way_count'] / topic_summary['n_resolutions']).round(1)
topic_summary['against_us_pct'] = (100 * topic_summary['against_us_count'] / topic_summary['n_resolutions']).round(1)
topic_summary['with_majority_pct'] = (100 * topic_summary['with_majority_count'] / topic_summary['n_resolutions']).round(1)

# Sort by 4-way alignment
topic_summary = topic_summary.sort_values('four_way_pct', ascending=False).reset_index(drop=True)
topic_summary['rank'] = range(1, len(topic_summary) + 1)

# Select display columns
display_cols = [
    'rank', 'meta_topic_label', 'n_resolutions',
    'four_way_pct', 'three_way_pct', 'two_way_pct',
    'against_us_pct', 'with_majority_pct'
]

display_df = topic_summary[display_cols].copy()
display_df.columns = [
    'Rank', 'Meta-Topic', 'N',
    '4-Way %', '3-Way %', '2-Way %',
    'vs US %', 'w/ Majority %'
]

print("=" * 100)
print("CRINK VOTING ALIGNMENT BY META-TOPIC (UNGA)")
print(f"Total resolutions: {len(resolution_df)}")
print("Sorted by 4-way alignment (all CRINK countries vote together)")
print("=" * 100)
print()
print(display_df.to_string(index=False))
print()
print("=" * 100)
print("\nColumn definitions:")
print("  4-Way %: Share of resolutions where all 4 CRINK vote the same")
print("  3-Way %: Share where 3 of 4 CRINK vote the same")
print("  2-Way %: Share where 2 of 4 CRINK vote the same")
print("  vs US %: Share where CRINK majority voted against US")
print("  w/ Majority %: Share where CRINK majority aligned with UN majority")

Generating topic alignment summary table...

CRINK VOTING ALIGNMENT BY META-TOPIC (UNGA)
Total resolutions: 2597
Sorted by 4-way alignment (all CRINK countries vote together)

 Rank                Meta-Topic    N  4-Way %  3-Way %  2-Way %  vs US %  w/ Majority %
    1        Economic Sanctions    1    100.0      0.0      0.0    100.0          100.0
    2 Israel/Palestine Conflict  298     76.2     22.8      1.0    100.0           99.7
    3 International Cooperation   50     76.0     14.0     10.0     84.0           92.0
    4               Development  146     73.3     12.3     12.3     87.7           93.8
    5              Displacement   61     54.1     11.5     34.4     96.7           60.7
    6     Human Rights in CRINK    4     50.0     50.0      0.0    100.0          100.0
    7             Miscellaneous 1511     49.7     33.6     15.8     83.4           83.5
    8              Human Rights  202     49.0     22.3     25.2     88.6           51.0
    9         International Law 

In [13]:
# Cell 13: Save Results

timestamp = datetime.now().strftime('%Y%m%d')

# Table naming for publication
TABLE_NAME = "Table2_Topic_Distribution_CRINK_Fully_Aligned"
TABLE_CAPTION = "Topic Distribution of Resolutions where CRINK States Fully Aligned in the UNGA Plenary, 1991-2024"

# Save summary table as CSV
summary_csv = results_dir / f'{TABLE_NAME}_UNGA_{timestamp}.csv'
topic_summary.to_csv(summary_csv, index=False)
print(f"✓ Saved: {summary_csv.name}")

# Save full resolution data with meta-topics
full_csv = results_dir / f'resolutions_with_metatopics_UNGA_{timestamp}.csv'
export_cols = ['undl_id', 'title', 'date', 'year', 'cluster', 'topic_label', 
               'meta_topic_label', 'meta_topic_id', 'crink_group_vote',
               'max_agreement_count', 'four_vote_alignment', 'three_vote_alignment',
               'two_vote_alignment', 'crink_against_us', 'crink_with_majority']
export_cols = [c for c in export_cols if c in resolution_df.columns]
resolution_df[export_cols].to_csv(full_csv, index=False)
print(f"✓ Saved: {full_csv.name}")

# Generate LaTeX table
def generate_latex_table(df_table, caption, label):
    """Generate LaTeX table with academic formatting."""
    df_clean = df_table.drop(columns=['Rank']) if 'Rank' in df_table.columns else df_table.copy()
    
    # Build LaTeX
    n_cols = len(df_clean.columns)
    col_spec = 'l' + 'r' * (n_cols - 1)
    
    latex = f"""\\begin{{table}}[htbp]
\\centering
\\footnotesize
\\begin{{tabular}}{{{col_spec}}}
\\hline
"""
    
    # Header
    headers = [c.replace('%', '\\%').replace('_', ' ') for c in df_clean.columns]
    latex += ' & '.join([f'\\textbf{{{h}}}' for h in headers]) + ' \\\\\n'
    latex += '\\hline\n'
    
    # Data rows
    for _, row in df_clean.iterrows():
        values = []
        for col in df_clean.columns:
            val = row[col]
            if isinstance(val, float):
                values.append(f'{val:.1f}')
            else:
                values.append(str(val).replace('&', '\\&').replace('%', '\\%'))
        latex += ' & '.join(values) + ' \\\\\n'
    
    latex += f"""\\hline
\\end{{tabular}}
\\caption{{{caption}}}
\\label{{{label}}}
\\end{{table}}
"""
    return latex

latex_table = generate_latex_table(display_df, TABLE_CAPTION, "tab:table2_topic_distribution")

latex_file = results_dir / f'{TABLE_NAME}_UNGA.tex'
with open(latex_file, 'w', encoding='utf-8') as f:
    f.write(latex_table)
print(f"✓ Saved: {latex_file.name}")

print(f"\n{'='*60}")
print("ALL RESULTS SAVED")
print(f"{'='*60}")
print(f"\nOutput files in: {results_dir}")
print(f"\nTable 2: {TABLE_CAPTION}")

✓ Saved: Table2_Topic_Distribution_CRINK_Fully_Aligned_UNGA_20260109.csv
✓ Saved: resolutions_with_metatopics_UNGA_20260109.csv
✓ Saved: Table2_Topic_Distribution_CRINK_Fully_Aligned_UNGA.tex

ALL RESULTS SAVED

Output files in: c:\Users\Lucian\OneDrive - Tulane University\01 IFSH\Data Sciences try\measuring-CRINK-alignment-UN\results

Table 2: Topic Distribution of Resolutions where CRINK States Fully Aligned in the UNGA Plenary, 1991-2024


## Summary

This notebook has:
1. Loaded UN voting data (5,415 unique resolutions, 1946-2024)
2. Generated or loaded embeddings using OpenAI's `text-embedding-3-large` model
3. Run HDBSCAN clustering to assign resolutions to topic clusters
4. Applied confirmed topic mappings using fuzzy matching to reuse existing meta-topic assignments
5. Assigned noise resolutions to meta-topics using undl_id-based mapping
6. Calculated CRINK voting alignment metrics (4-way, 3-way, 2-way, vs US, with UN majority)
7. Generated summary tables for publication

### Mapping Reuse Strategy
- **Topic mappings**: Fuzzy word-overlap matching to map new cluster labels → existing meta-topics
- **Noise mappings**: Direct undl_id lookup (works for overlapping resolutions)
- **Unmatched items**: Assigned to "Miscellaneous" category

### Reproducibility Notes
- Embeddings are saved to `data/mappings/UNGA_embeddings.npy`
- HDBSCAN parameters: `min_cluster_size=10`, `min_samples=1`
- Requires `OPEN_AI_API` environment variable for embedding generation